In [11]:
import pandas as pd
import gzip
from datetime import datetime
import shutil
import os

def load_gzipped_tsv_in_chunks(file_path, chunk_size=10000):
    chunks = []
    for chunk in pd.read_csv(file_path, delimiter='\t', encoding='utf-8', chunksize=chunk_size):
        chunks.append(chunk)
    return pd.concat(chunks, ignore_index=True)

# Note: The large input file should be downloaded from https://datasets.imdbws.com/
input_file = 'imdb_flat_files/title.basics.tsv.gz'
output_file = 'imdb_flat_files/filtered_title_basics.tsv.gz'

# Read the compressed TSV file
df = load_gzipped_tsv_in_chunks(input_file)

# Get the current year
current_year = datetime.now().year
# Calculate the threshold year
threshold_year = current_year - 10

# Convert 'startYear' to numeric (Int64), forcing errors to NaN
df['startYear'] = pd.to_numeric(df['startYear'], errors='coerce').astype('Int64')
filtered_df = df[(df['startYear'] >= threshold_year) & (df['startYear'].notna())]

# Normalize column names for consistency
print(filtered_df.head())

rename_cols = {
    'tconst': 'movie_id',
    'startYear': 'start_year', 
    'titleType': 'title_type',
    'originalTitle': 'original_title',
    'primaryTitle': 'primary_title',
    'isAdult': 'is_adult',
    'endYear': 'end_year',
    'runtimeMinutes': 'runtime_minutes',
    'genres': 'genres'
}
filtered_df.rename(columns=rename_cols, inplace=True)

filtered_df = filtered_df[filtered_df['title_type'] == 'movie']

# Convert runtime_minutes and end_year to numeric and handle missing values
filtered_df['runtime_minutes'] = pd.to_numeric(filtered_df['runtime_minutes'], errors='coerce').astype('float64')
filtered_df['end_year'] = pd.to_numeric(filtered_df['end_year'], errors='coerce').astype('Int64')

# Reload filtered data for merging
title_basics = filtered_df
title_ratings = load_gzipped_tsv_in_chunks('imdb_flat_files/title.ratings.tsv.gz')
title_ratings.rename(columns={'tconst': 'movie_id', 'averageRating': 'averagerating', 'numVotes': 'numvotes'}, inplace=True)
print(title_ratings.head())

# Merge the filtered movies with the ratings data on movie_id
merged_df = pd.merge(filtered_df, title_ratings, on='movie_id', how='left')
merged_df['numvotes'] = pd.to_numeric(merged_df['numvotes'], errors='coerce').astype('Int64')

# Display merged dataframe info
print(merged_df.info())

# Save the filtered DataFrame to a TSV file
filtered_tsv_file = 'filtered_title_basics.tsv'
merged_df.to_csv(filtered_tsv_file, sep='\t', index=False, encoding='utf-8')

# Compress the TSV file
with open(filtered_tsv_file, 'rb') as f_in:
    with gzip.open(output_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

# Optionally, remove the intermediate TSV file
os.remove(filtered_tsv_file)
print(f"Filtered and compressed file saved as {output_file}")

threshold_year = current_year - 3
years_df = merged_df[(merged_df['start_year'] >= threshold_year) & (merged_df['start_year'].notna())]
print(years_df.head())

# Save the resulting DataFrame as a pickle file
years_df.to_pickle('last_three_years_movies_with_ratings.pkl')


          tconst titleType                      primaryTitle  \
11634  tt0011801     movie                  Tötet nicht mehr   
13079  tt0013274     movie       Istoriya grazhdanskoy voyny   
55749  tt0056840     short                          Aufsätze   
56272  tt0057369     short  Number 14: Late Superimpositions   
59194  tt0060361     short                          EMS nr 1   

                          originalTitle isAdult  startYear endYear  \
11634                  Tötet nicht mehr       0       2019      \N   
13079       Istoriya grazhdanskoy voyny       0       2021      \N   
55749                          Aufsätze       0       2021      \N   
56272  Number 14: Late Superimpositions       0       2023      \N   
59194                          EMS nr 1       0       2016      \N   

      runtimeMinutes        genres  
11634             \N  Action,Crime  
13079             94   Documentary  
55749             10         Short  
56272             30         Short  
59194    

/tmp/ipykernel_854902/2480158035.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.rename(columns=rename_cols, inplace=True)


    movie_id  averagerating  numvotes
0  tt0000001            5.7      2058
1  tt0000002            5.7       276
2  tt0000003            6.5      2022
3  tt0000004            5.4       179
4  tt0000005            6.2      2786
<class 'pandas.core.frame.DataFrame'>
Int64Index: 195091 entries, 0 to 195090
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         195091 non-null  object 
 1   title_type       195091 non-null  object 
 2   primary_title    195091 non-null  object 
 3   original_title   195091 non-null  object 
 4   is_adult         195091 non-null  object 
 5   start_year       195091 non-null  Int64  
 6   end_year         0 non-null       Int64  
 7   runtime_minutes  145508 non-null  float64
 8   genres           195091 non-null  object 
 9   averagerating    101899 non-null  float64
 10  numvotes         101899 non-null  Int64  
dtypes: Int64(3), float64(2), object(6)
memory usa

In [8]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 195091 entries, 0 to 195090
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         195091 non-null  object 
 1   title_type       195091 non-null  object 
 2   primary_title    195091 non-null  object 
 3   original_title   195091 non-null  object 
 4   is_adult         195091 non-null  object 
 5   start_year       195091 non-null  Int64  
 6   end_year         0 non-null       Int64  
 7   runtime_minutes  145508 non-null  float64
 8   genres           195091 non-null  object 
 9   average_rating   101899 non-null  float64
 10  num_votes        101899 non-null  Int64  
dtypes: Int64(3), float64(2), object(6)
memory usage: 18.4+ MB
